# breweryBizdat
### get business data about the brewery

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [5]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM breweries", con=engine)
brew_df

,index,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode
0,0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403
1,1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605
2,2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732
3,3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701
4,4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801
262,331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792
263,332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906
264,333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587


In [6]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
place_url = "https://maps.googleapis.com/maps/api/place/details/json"

In [7]:
#create columns for new data from api call
brew_df['google_rating']=''
brew_df['google_numratings']=''
brew_df

,index,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode,google_rating,google_numratings
0,0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403,,
1,1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605,,
2,2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732,,
3,3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701,,
4,4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801,,
262,331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792,,
263,332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906,,
264,333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587,,


In [8]:
#hit google places api

for index, row in brew_df.iterrows():
    params = {
        "key": gkey,
        "query": f"brewery",
        "location":f"{row.latitude},{row.longitude}"
    }
    
    response = requests.get(base_url, params=params)
    place = response.json()['results'][0]
        
    try:
        brew_df.at[index, 'google_rating'] = place['rating']
        brew_df.at[index, 'google_numratings'] = place['user_ratings_total']

    except:
        pass

In [9]:
brew_df

,index,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode,google_rating,google_numratings
0,0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403,3.5,4
1,1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605,4.2,241
2,2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732,4.7,206
3,3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701,4.2,6
4,4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348,4.7,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801,4.2,687
262,331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792,4.7,166
263,332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906,4.4,696
264,333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587,4.6,185


In [10]:
# add data to sqlite database
brew_df.to_sql('breweries', con=engine, if_exists='replace')

In [11]:
#test sqlite data
engine.execute("SELECT * FROM breweries").fetchall()[0]

(0, 0, 5016, 'Bark Brewing Company', 'brewpub', '3021 Spring Garden St', 'Greensboro', 'North Carolina', '27403-1968', 'United States', '-79.8436043406271', '36.0620103554184', '3368561406', 'http://barkbrewingcompany.com', '2018-08-24T15:41:02.410Z', '27403', '3.5', 4)